# Python私房手册-用pytest进行测试

先简单的描述一下使用`pytest`进行测试的整个过程：首先，我们编写一些专门用来进行测试的代码，这些代码被称为测试用例，代码所在的文件被称为测试文件，测试文件和测试用例都需要遵循`pytest`的命名规则，然后在命令行中直接运行`pytest`，`pytest`会搜索测试文件和测试用例并运行，这个过程被称为测试搜索，最后给出测试结果。  

## 编写基本的测试

### 使用`assert`声明

### 预期异常

### 参数化测试

## 如何运行测试

1. 直接使用`pytest`加命令行选项的方式：  
如果不提供任何参数，`pytest`会在当前目录以及子目录下寻找所有测试用例，不过，`pytest`可以选择运行某个目录、文件、类中的测试，还可以根据测试用例的名称或者根据标记来运行测试。
2. 使用`python -m pytest`的方式：  
这种方式和上一种几乎一样，唯一的区别就是它会把你当前的路径加入到`sys.path`中去。

### 单个目录、文件和类

- 以目录做为`pytest`的参数即可。如：`pytest tests`。
- 路径名加文件名。如：`pytest tests/testfile.py`。
- 单个类或方法：在文件名后加上`::`符号和类名或方法名，如：`pytest tests/testfile.py::TestCase`。
- 测试类中的方法：同样在类后加上`::`符号和方法名称，如：`pytest tests/testfile.py::TestCase::testfunc`。

简单来说，文件级别以上填写完整路径即可，文件内部使用`::`符号区分级别。

### 使用`-k`参数根据名称选取子集

`-k`选项可以根据名称来选取子集，后面接表达式，表达式中可以包含"and, or, not"，如下：
```python
pytest -k _raise # 选取测试名称中包含_raise字段的所有测试项目
pytest -k "_raise not delete" # 选取测试名称中包含_raise但是不包含delete字段的所有测试
```
注意：`-k`如果是单个参数，可以不加引号，加引号的话只能加双引号，单引号不识别。

### 使用`-m`参数根据标记选取子集

`-m`选项可以根据标记来选取子集，如：
```python
pytest -m smoke tests
```
此时只有通过`@pytest.mark.smoke`装饰器标记了的测试用例才会运行。要注意的是，这里`smoke`是自定义的标记，自定义标记需要先注册，有两种方法：
1. 在`pytest.ini`中配置，格式如下：
```ini
[pytest]
markers = smoke: mark smoke test case
```
注意格式，也可以写成这样：
```ini
[pytest]
markers = 
    smoke: mark smoke test case
```
2. 在`confest.py`文件中注册，格式如下：
```python
def pytest_configure(config):
    config.addinivalue_line("markers", "smoke: This is a customized test mark")
```
点击查看官网关于[如何注册自定义标记](https://docs.pytest.org/en/latest/mark.html)的说明，另外`pytest.ini`和`confest.py`放在包含测试用例的文件夹根目录下。`@pytest.mark`有四个内置的标记，分别是`skip`,`skipif`,`xfail`,`parametrize`，单独分章节学习。

### 常用的命令行选项

- `--collect-only`：展示哪些测试用例会被运行，仅仅只是展示，并不实际执行测试用例。
- `-x`：遇到失败即停止，不再运行剩下的测试用例。
- `-r char`：`-r`后面接字符，比如`sxX`，分别表示被跳过，预计失败和预计失败但是通过的测试用例。默认情况下，对于上面几种情况，是没有输出报告的，只有加了`-r`，才会单独的给出报告，另外，可以`-r sxX`，也可以不要空格，直接`-rsxX`。
- `--maxfail=num`：允许失败几次再停止。
- `-s`：关闭`capsys`，即将`print`输出到`stdout`。
- `-v`：输出的信息更详细，和不带`v`最明显的区别是，不带`v`的话，输出结果时，每个测试文件占一行，带`v`，输出结果时，每个测试用例占一行。
- `-q`：简化输出信息，和`--tb=line`搭配适用，可以只打印异常的代码位置。不显示一开始的`rootdir`，`platform`,`plugins`等信息。
- `--lf`：这个选项一开始没太理解，后来才搞懂，它其实分两步，先把所有测试用例运行一遍，然后再把那些出错的再运行一遍，对的就不再运行了，只给出第二遍运行的结果。
- `--ff`：这个选项也是运行两遍，第两遍先运行那些错误的测试用例，给出第二遍运行的结果。
- `-l`：输出信息还会给出错误的测试用例中的局部变量。
- `--tb=style`：配置测试失败是的回溯信息的显示格式，有6种：
 - `--tb=auto`：默认值，仅打印第一个和最后一个用例的回溯信息。
 - `--tb=long`：输出最详尽的回溯信息。
 - `--tb=line`：只告诉我们错误的位置，对于大量测试用例的情况下很有用，可以用它发现失败的共性。
 - `--tb=short`：仅输出`assert`一行以及系统判别内容（不包含上下文）。
 - `--tb=no`：屏蔽所有回溯信息。
 - `--tb=native`：只输出标准库的回溯信息，不显示额外信息。
- `--duration=N`：统计测试过程中哪几个阶段是最慢的。`--duration=0`的话会根据耗时从长到短排列。
- `--setup-show`：回溯显示整个`fixture`的执行过程。
- `--fixtures`：列出所有可用的`fixtures`，包括他们的位置（哪个文件哪一行），以及`docstring`。
- `--markers`：查看所有可用的`marker`，包含内置的和自定义的。
- `--doctest_modules`：搜索并运行`doctest`测试用例。
- `--strict`：改变`pytest`输出报告的某些默认行为，后面还可以接其它选项，比如`--strict-markers`，未注册的`marker`抛出错误。

除了上面列出来的常用的一些，还有很多选项，可以通过`--help`查看，另外，感觉`--lf`和`--ff`两个选项好像用处不大，可能是还没有遇到适用的场景。

## 使用`Fixture`满足更多测试需求

### 什么是`Fixture`

简单来说，`fixture`就是一个函数，`pytest`会先判断测试函数的参数是不是一个`fixture`，如果是，则会在运行测试函数之前先去执行这个`fixture`函数，写`Fixture`很简单，适用`@pytest.fixture()`（注意，有括号）装饰器装饰任何一个函数即可，如下：
```python
@pytest.fixture()
def some_date():
    return 42
```
现在，就有了一个名为`some_data`的`fixture`，你可以在测试函数中使用它：
```python
def test_some_data(some_data):
    assert some_data == 42
```
这些`fixture`写在哪里呢，可以和测试函数写在一起，也可以单独写到一个名为`conftest.py`的文件中，`pytest`会先判断测试函数的参数是不是一个`fixture`，它会先搜索测试函数所在的模块，如果没找到，再到`conftest.py`这个文件中去查找。

### 理解测试用例执行过程

了解了什么是`Fixture`，就可以理解一个测试用例的完整的执行过程。一般执行一个测试用例，分为三个阶段：
- `SETUP`：此时`pytest`发现测试用例中的参数是一个`fixture`，则开始执行`fixture`函数，执行到`yield`语句的时候就停止。
- `CALL`：开始执行测试用例。
- `TEARDOWN`：测试用例执行完毕，又回到`fixture`函数中，执行`yield`语句的后续部分。

通过`--setup-show`命令行参数可以清楚的看到这个过程，`--durations=num`命令行参数可以告诉你，哪几个阶段耗时最长。

有两种方式可以执行`Teardown`操作，一种是使用`yield`，一种是为`request`添加`addfinalizer`，但是两者有细微的差别：
```python
import pytest


@pytest.fixture
def f1():
    print('fixture 1 start!')
    yield 42
    raise ValueError('something wrong happened!')
    print('do some clean work!')
    print('fixture 1 end!')


@pytest.fixture
def f2(request):
    print('fixture 2 start!')

    def clean_work():
        print('do some clean work!')
        print('fixture 2 end!')

    request.addfinalizer(clean_work)

    raise ValueError('something wrong happened!')
    return 42


def test_fixture_1(f1):
    print(f1)


def test_fixture_2(f2):
    print(f2)
```
对于`test_fixture_1`，抛出错误之后的代码不会执行。但是对`test_fixture_2`，只要`addfinalizer`被添加到`request`上，那么不管之后是不是有异常，回调函数都会执行。`request`是内置的`fixture`，可以理解为调用`fixture`的测试函数。

另外，如果一个测试函数包含多个包含`Teardown`代码的`fixture`，其中一个`fixture`抛出了异常，不会影响其它的`fixture`：
```python
import pytest


@pytest.fixture
def f1():
    print('fixture 1 start')
    yield 42
    print('fixture 1 end')


@pytest.fixture
def f2():
    print('fixture 2 start')
    raise ValueError('f2 error')
    yield 33
    print('fixture 2 end')


def test_func(f1, f2):
    print('test function start:', f1, f2)
```
`f2`这个`fixture`抛出了异常，测试函数不会执行，`f2`异常后的代码不会执行，但是`f1`会正常执行完毕，打印信息为：
```
fixture 1 start
fixture 2 start
fixture 1 end
```
所以，`fixture`因为仅只完成一个功能点，尽量为原子操作。

### fixture可用性

一个`fixture`是否可用，是看它的`scope`，但是是以测试函数的角度去看的，比如：
```python
import pytest


@pytest.fixture
def order():
    return []


@pytest.fixture
def outer(order, inner):
    order.append("outer")


class TestOne:
    @pytest.fixture
    def inner(self, order):
        order.append("one")

    def test_order(self, order, outer):
        assert order == ["one", "outer"]


class TestTwo:
    @pytest.fixture
    def inner(self, order):
        order.append("two")

    def test_order(self, order, outer):
        assert order == ["two", "outer"]
```
当调用`test_order`，会依次执行`order`和`outer`，执行`outer`时，接收`inner`，显然`inner`对于`outer`而言，是不可见的。但实际上，可见不可见是对测试函数而言的，对于`test_order`来说，`inner`是可见的，所以在执行`outer`这个`fixture`时，`inner`这个`fixture`会正常执行。

### fixture搜索顺序

在包结构中，fixture会一直向上搜索，只要文件夹包含`__init__.py`，pytest就会认为是测试路径的一部分，一直搜索到最后一个包含`__init__.py`的文件夹，注意几点：
1. 只会向上搜索，不会向下搜索。
2. 同一个目录内的文件夹和文件视为同级的范围
3. 一个目录下的conftest.py文件里的fixture对当前范围内的所有文件可见。

### fixture的初始化顺序

fixture的初始化顺序和它的定义位置，执行顺序什么的都没有关系。
1. 由它的范围决定，简单来说，session->package->module->class->function
2. 相同范围的fixture由依赖关系决定初始化顺序。
3. 自动运行的fixture在其范围内会首先运行。
4. 自动运行的fixture a请求的fixture b也会成为自动运行的，不过a被使用的时候才会这样。

### 通过request获取测试上下文

通过内置的fixture`request`可以获取测试上下文，比如：
- `request.module`获取测试函数所在模块：
    ```python
    import pytest
    import smtplib

    @pytest.fixture(scope="module")
    def smtp_connection(request):
        server = getattr(request.module, "smtpserver", "smtp.gmail.com")
        smtp_connection = smtplib.SMTP(server, 587, timeout=5)
        yield smtp_connection
        print("finalizing {} ({})".format(smtp_connection, server))
        smtp_connection.close()
    ```
    测试函数：
    ```python
    smtpserver = "mail.python.org"  # will be read by smtp fixture

    def test_showhelo(smtp_connection):
        assert 0, smtp_connection.helo()
    ```
- 获取marker传递的值，node不是很清楚作用，待补充：
    ```python
    import pytest

    @pytest.fixture
    def fixt(request):
        marker = request.node.get_closest_marker("fixt_data")
        if marker is None:
            # Handle missing marker in some way...
            data = None
        else:
            data = marker.args[0]

        # Do something with the data
        return data

    @pytest.mark.fixt_data(42)
    def test_fixt(fixt):
        assert fixt == 42
    ```

### fixture工厂

fixture可以返回一个函数，动态产出数据：
```python
@pytest.fixture
def make_customer_record():
    def _make_customer_record(name):
        return {"name": name, "orders": []}

    return _make_customer_record


def test_customer_records(make_customer_record):
    customer_1 = make_customer_record("Lisa")
    customer_2 = make_customer_record("Mike")
    customer_3 = make_customer_record("Meredith")
```

### 参数化fixture

fixture可以参数化：
```python
# content of conftest.py
import pytest
import smtplib


@pytest.fixture(scope="module", params=["smtp.gmail.com", "mail.python.org"])
def smtp_connection(request):
    smtp_connection = smtplib.SMTP(request.param, 587, timeout=5)
    yield smtp_connection
    print("finalizing {}".format(smtp_connection))
    smtp_connection.close()
```
调用`smtp_connection`的测试函数相当于会运行2次，参数可以通过`request.param`获取。

默认情况下，打印测试项的时候，会在测试函数后添加`param`的值以示区分，可以通过`ids`参数指定名称，`ids`还可以接受一个函数，动态定义名称：
```python
# content of test_ids.py
import pytest


@pytest.fixture(params=[0, 1], ids=["spam", "ham"])
def a(request):
    return request.param


def test_a(a):
    pass


def idfn(fixture_value):
    if fixture_value == 0:
        return "eggs"
    else:
        return None


@pytest.fixture(params=[0, 1], ids=idfn)
def b(request):
    return request.param


def test_b(b):
    pass
```
上面例子加`--collect-only`运行（表示只收集打印测试项，不运行），结果如下：
```
$ pytest --collect-only
=========================== test session starts ============================
platform linux -- Python 3.x.y, pytest-6.x.y, py-1.x.y, pluggy-1.x.y
cachedir: $PYTHON_PREFIX/.pytest_cache
rootdir: $REGENDOC_TMPDIR
collected 10 items

<Module test_anothersmtp.py>
  <Function test_showhelo[smtp.gmail.com]>
  <Function test_showhelo[mail.python.org]>
<Module test_ids.py>
  <Function test_a[spam]>
  <Function test_a[ham]>
  <Function test_b[eggs]>
  <Function test_b[1]>
<Module test_module.py>
  <Function test_ehlo[smtp.gmail.com]>
  <Function test_noop[smtp.gmail.com]>
  <Function test_ehlo[mail.python.org]>
  <Function test_noop[mail.python.org]>

======================= 10 tests collected in 0.12s ========================
```

### fixture参数中包含marks

`Pytest.param()`可用于在参数化fixture的值集中应用`marks`，其方式与`@pytest.mark.parameterize`使用`marks`的方式相同:
```python
import pytest


@pytest.fixture(params=[0, 1, pytest.param(2, marks=pytest.mark.skip)])
def data_set(request):
    return request.param


def test_data(data_set):
    pass
```
运行的时候可以看到，参数2被忽略：
```
$ pytest test_fixture_marks.py -v
=========================== test session starts ============================
platform linux -- Python 3.x.y, pytest-6.x.y, py-1.x.y, pluggy-1.x.y -- $PYTHON_PREFIX/bin/python
cachedir: $PYTHON_PREFIX/.pytest_cache
rootdir: $REGENDOC_TMPDIR
collecting ... collected 3 items

test_fixture_marks.py::test_data[0] PASSED                           [ 33%]
test_fixture_marks.py::test_data[1] PASSED                           [ 66%]
test_fixture_marks.py::test_data[2] SKIPPED (unconditional skip)     [100%]

======================= 2 passed, 1 skipped in 0.12s =======================
```

### 根据fixture实例自动分组测试

Pytest在测试运行期间最小化活动fixture的数量。如果有一个参数化的fixture，所有的测试会先用第一个参数创建一个fixture实例，运行使用了该实例的所有测试项，然后终结这个fixture，再用第二个参数创建第二个fixture实例，运行与该实例有关的测试项。相当于根据fixture的参数对所有测试项进行了分组。
```python
import pytest


@pytest.fixture(scope="module", params=["mod1", "mod2"])
def modarg(request):
    param = request.param
    print("  SETUP modarg", param)
    yield param
    print("  TEARDOWN modarg", param)


@pytest.fixture(scope="function", params=[1, 2])
def otherarg(request):
    param = request.param
    print("  SETUP otherarg", param)
    yield param
    print("  TEARDOWN otherarg", param)


def test_0(otherarg):
    print("  RUN test0 with otherarg", otherarg)


def test_1(modarg):
    print("  RUN test1 with modarg", modarg)


def test_2(otherarg, modarg):
    print("  RUN test2 with otherarg {} and modarg {}".format(otherarg, modarg))
```
仔细观察输出：
```
$ pytest -v -s test_module.py
=========================== test session starts ============================
platform linux -- Python 3.x.y, pytest-6.x.y, py-1.x.y, pluggy-1.x.y -- $PYTHON_PREFIX/bin/python
cachedir: $PYTHON_PREFIX/.pytest_cache
rootdir: $REGENDOC_TMPDIR
collecting ... collected 8 items

test_module.py::test_0[1]   SETUP otherarg 1  # otherarg的scope是函数，因此每次执行测试函数都会执行otherarg fixture
  RUN test0 with otherarg 1
PASSED  TEARDOWN otherarg 1

test_module.py::test_0[2]   SETUP otherarg 2
  RUN test0 with otherarg 2
PASSED  TEARDOWN otherarg 2

# modarg的scope是模块，测试才开始创建
test_module.py::test_1[mod1]   SETUP modarg mod1  
  RUN test1 with modarg mod1
PASSED

# 参数为mod1的modarg这个fixture还被test2使用，所以先执行test2，而不是继续执行下一个test_1
test_module.py::test_2[mod1-1]   SETUP otherarg 1  
  RUN test2 with otherarg 1 and modarg mod1
PASSED  TEARDOWN otherarg 1

test_module.py::test_2[mod1-2]   SETUP otherarg 2
  RUN test2 with otherarg 2 and modarg mod1
PASSED  TEARDOWN otherarg 2

test_module.py::test_1[mod2]   TEARDOWN modarg mod1
  SETUP modarg mod2
  RUN test1 with modarg mod2
PASSED
test_module.py::test_2[mod2-1]   SETUP otherarg 1
  RUN test2 with otherarg 1 and modarg mod2
PASSED  TEARDOWN otherarg 1

test_module.py::test_2[mod2-2]   SETUP otherarg 2
  RUN test2 with otherarg 2 and modarg mod2
PASSED  TEARDOWN otherarg 2
  TEARDOWN modarg mod2


============================ 8 passed in 0.12s =============================
```
可见，上面的测试函数执行的顺序为0->0->1->2->2->1->2->2，而不是0->0->1->1->2->2。modarg是模块范围，2个参数，通过调整执行顺序，这个fixture一共执行了2次。而不是test1执行2次，test2执行2次。

之前这一点理解有误，以为如果fixture是模块范围，则会在脚本运行最初就执行一次fixture，结束才调用finalizer终止。实际是引用它的测试函数执行才会创建这个fixture，然后与这个fixture有关的其它的测试函数依次运行，最后调用finalizer终止这个fixture。

### 通过`usefixtures`在类和模块中使用fixture

有时候fixture不需要当作参数传递给测试方法，此时在类或者模块中，我们可以使用`pytest.mark.usefixtures`来指定所需的fixture，而不需要每个测试方法都把`fixture`当成参数进行传递。
```python
import os
import shutil
import tempfile

import pytest


@pytest.fixture
def cleandir():
    old_cwd = os.getcwd()
    newpath = tempfile.mkdtemp()
    os.chdir(newpath)
    yield
    os.chdir(old_cwd)
    shutil.rmtree(newpath)
```
可以直接装饰一个类，则每个测试方法都会执行该fixture。
```python
# content of test_setenv.py
import os
import pytest


@pytest.mark.usefixtures("cleandir")
class TestDirectoryInit:
    def test_cwd_starts_empty(self):
        assert os.listdir(os.getcwd()) == []
        with open("myfile", "w") as f:
            f.write("hello")

    def test_cwd_again_starts_empty(self):
        assert os.listdir(os.getcwd()) == []
```
在模块范围内，可以直接该语句：
```python
pytestmark = pytest.mark.usefixtures("cleandir")
```
设置，可以在配置文件`.ini`中使用，此时所有测试用例都会执行该fixture，而不需要将这个fixture当成参数传递给测试方法才会执行fixture：
```python
# content of pytest.ini
[pytest]
usefixtures = cleandir
```

### 覆盖不同级别的fixture

当fixture和本地参数同名时，本地参数会覆盖掉fixture:
```python
tests/
    __init__.py

    conftest.py
        # content of tests/conftest.py
        import pytest

        @pytest.fixture
        def username():
            return 'username'

        @pytest.fixture
        def other_username(username):
            return 'other-' + username

    test_something.py
        # content of tests/test_something.py
        import pytest
        
        # 当参数和fixture同名时，优先参数
        @pytest.mark.parametrize('username', ['directly-overridden-username'])
        def test_username(username):
            assert username == 'directly-overridden-username'
        
        # 注意，注意，other_username是fixture，此时它的参数本地参数username而不是fixture username
        # 正常情况下，定义了本地参数，则一定要传入测试函数，此时相当于通过fixture传入，不会报错
        @pytest.mark.parametrize('username', ['directly-overridden-username-other'])
        def test_username_other(other_username):
            assert other_username == 'other-directly-overridden-username-other'
```

可以通过设置`indirect`参数改变这种行为，此时使用fixture，忽略本地参数：
```python
import pytest


@pytest.fixture()
def username():
    return 'shl'


@pytest.mark.parametrize('username', ['telecomshy'], indirect=['username'])
def test_username(username):
    assert username == 'shl'
```

如果使用了`request.param`，情况又不一样了。我们在fixture参数化的时候展示过，当fixture包含`params`参数时，`request.param`总是`fixture`的参数：
```python
@pytest.fixture(params=['shl'])
def f(request):
    return request.param


def test_f(f):
    assert f == 'shl'
```
即使测试函数包含本地参数，`request.param`仍然是fixture的参数：
```python
@pytest.fixture(params=['shl'])
def f(request):
    return request.param


@pytest.mark.parametrize('q', ['shy'])
def test_f(q, f):
    assert f == 'shl'
```
但是当测试函数的本地参数和fixture同名，且本地参数包含`indirect`关键字时，`request.param`却变成了本地参数...
```python
@pytest.fixture(params=['shl'])
def f(request):
    return request.param


@pytest.mark.parametrize('f', ['shy'], indirect=True)
def test_f(f):
    assert f == 'shy'
```

### 使用其它项目的fixture

pytest以项目为入口点，一般只能看到本项目内confest.py文件定义的fixture。如果想要看到别的项目的fixture，假设在`mylibrary.fixtures`中有一些`fixture`，现在想重用它们到`app/tests`目录。可以在`app/tests/conftest.py`中定义指向该模块的`pytest_plugins`:
```python
pytest_plugins = "mylibrary.fixtures"
```

### 自定义`Fixture`

#### 通过`conftest.py`文件共享`fixture`

`fixture`可以单独写在一个名为`conftest.py`文件中，`conftest.py`文件中的所有`fixture`可以供所在目录以及子目录下的所有测试用例使用。要注意一点，尽管`conftest.py`是`python`模块，但是它不能被导入。

#### `Fixture`主要使用场景

`fixgure`主要有两个使用场景：
1. 使用`fixture`执行配置及销毁。
比如测试需要用到数据库，那么可以写一个这样的`fixture`：
```python
@pytest.fixture()
def connect_db():
    start_db() # 执行数据库连接
    yield
    stop_db() # 执行数据库断开
```
这样就不用每个测试用例都写数据库连接和断开的代码。
2. 使用`fixture`传递测试数据。
如果有多个测试用例都要使用到的数据，此时可以把这些数据放到一个`fixture`里，如：
```python
@pytest.fixture()
def some_data():
    return "some data"
```

#### 指定`fixture`的作用范围

难道每次执行测试用例的时候，参数包含的`fixture`都会执行一次吗？默认情况下确实是这样，但是，我们可以通过`fixture`的`scope`参数来指定`fixture`的作用范围，一共有4个级别：
- `session`级别：当一次会话的时候执行一次，会话可以理解成运行一次`pytest`。
- `module`级别： 一个模块执行一次。
- `class`级别： 一个类一个级别。
- `function`级别： 函数级别，默认级别。

这里有几个问题要弄清楚：
1. **如果没有向测试用例的参数中传递`fixture`，`fixture`还会运行吗？**  
答案是否。不过如果在定义`fixture`时，设置了`autouse=True`，那么，不管有没有传递`fixture`，它也会根据定义时设定的`scope`级别自动运行，比如`session`级别的，每次会话运行一次，`function`级别的，每个测试用例都会运行一次。


2. **如果设置了`autouse=True`自动运行，那么在测试用例里使用某个已经运行的`fixture`，还需要通过参数传递给测试用例吗？**  
答案是肯定的，虽然`fixture`已经运行了，但是必须通过参数传递给测试用例才能使用，否则会抛出`NameError`。

#### 通过`@pytest.mark.usefixtures()`指定要使用的`fixture`

如果测试用例是函数，基本上用不上这个装饰器，只有测试用例是类的情况下，才有意义：
```python
@pytest.fixture()
def use_db():
    print('start_db')
    yield
    print('end_db')


@pytest.mark.usefixtures('use_db')
class TestCase:
    def test_something(self):
        assert 1
```
注意，使用`usefixtures`添加`fixture`，只适合用来执行配置和销毁，即没有任何返回值，因为如果要使用`usefixtures`方法添加的`fixture`，仍然要通过传参的方式将`fixture`传递给类中的方法，这样的话就没必要使用`usefixtures`添加`fixture`了。还是上面的例子:
```python
class TestCase:
    def test_something(self, use_db):
        assert 1
```
此时，既然已经将`use_db`传递给`test_something`，就没必要再使用`usefixtures`了，即`usefixtures`只适合指定无返回值，即测试类中的方法不需要使用到的执行配置和销毁的`fixture`。  
当然，也可以完全不用`usefixtures`，那么你得这样写：
```python
@pytest.fixture(scope='class')
def use_db():
    print('start_db')
    yield
    print('\nend_db')


class TestCase:
    def test_something(self, use_db):
        assert 1

    def test_other(self):
        assert 2
```
这样的写法很不友好，除非你将`use_db`得`autouse`设置为`True`，但是这样的话每个测试类都会执行`use_db`。所以，如果类级别需要执行配置和销毁的`fixture`，最好就是使用`usefixtures`方法。

#### 重命名`fixture`

默认情况下，`fixture`的名字就是`fixture`函数的名字，但是有时候，函数名字很长，此时可以通过`name`参数指定一个名称。这个很好理解，只是有个小技巧，可以通过`--fixtures`参数列出所有可用的`fixture`，包含内置的和自定义的，不过此时，只会以更改后的名字进行显示。

#### `fixture`的参数化

前面使用`@pytest.mark.parametrize()`参数化，可以一次将多个数据以参数形式传递给测试用例，但如果这些数据需要反复使用怎么办？此时可以使用`fixture`的`params`参数对数据进行参数化：
```python
origin_datas = ('data1', 'data2', 'data3')
processed_datas = ('Data1', 'Data2', 'Data3')


@pytest.fixture(params=zip(origin_datas, processed_datas))
def data(request):
    return request.param


def test_data(data):
    origin_data, processed_data = data
    assert origin_data.capitalize() == processed_data
```
`request`是`pytest`内置的一个`fixture`，其`param`属性代表`params`列表中的一个元素。和普通的参数化一样，可以通过`ids`指定参数名称，注意，`ids`可以指定一个函数，该函数将作用于`params`列表中的每一个元素，生成最终的标识。

#### 总结

最后对`@pytest.fixture`的几个重要的参数总结一下：
- `scope`：指定作用范围。
- `name`：重命名`fixture`。
- `params`：参数化`fixture`。
- `ids`：指定参数化`fixture`的标识。

### 内置`Fixture`

#### `tmpdir`和`tmpdir_factory`

这两个内置的`fixture`负责在测试开始运行前创建临时文件目录或者文件，并在测试后删除。区别是`tmpdir`是函数级别的，`tmpdir_factory`是会话级别的。注意：
- `tmpdir`本身是一个`py.path.local`类型的对象，而`tempdir_factory`是`_pytest.tmpdir.TempdirFactory`类型的对象，`tempdir_factory`的`mktemp()`方法返回的才是`py.path.local`对象，`py`是`python`的一个库，`py.path.local`对象是对本地文件系统的一个包装，可以方便的对文件夹和文件进行各种操作，官方文档[点这里](https://py.readthedocs.io/en/latest/path.html)。
- 如果要模块级别或者类级别的临时目录，可以使用`tmpdir_factory`自己再创建一个`fixture`。

#### `pytestconfig`

会话级别的`fixture`，它可以通过命令行参数、选项、配置文件等等来控制`pytest`，这个`fixture`其实是返回一个`_pytest.config.Config`对象，对象的官方文档[点这里](https://docs.pytest.org/en/latest/reference.html?highlight=pytestconfig#_pytest.fixtures.pytestconfig)，主要是写插件时用的多。

#### `cache`

`cache`是`session`级别的`fixture`，可以理解成类似`cookie`一样的东西。用于在测试会话之间保存一些信息。它有两个方法：
```python
cache.get(key, default)
cache.set(key, value)
```
分别来读取和设置信息，可以通过`--cache-clear`和`--cache-show`分别清空和显示`cache`，以下几点要注意：
1. `--cache-clear`会在每次`session`开始之前清空`cache`，而不是之后，后面接测试用例，因此总会保留一次运行的结果。
2. `--cache-show`命令行参数只是用来显示`cache`的内容，不会运行任何测试用例，它后面接的也不是测试用例，而是`cache`里面的`key`。

举个例子：
```python
def test_cache(cache):
    nums = cache.get('nums', None)
    if nums is None:
        cache.set('nums', 1)
    else:
        nums += 1
        cache.set('nums', nums)
    assert 1
```
查看一共运行了几次测试，首先运行测试用例：`pytest --cache-clear test_funcs.py::test_cache`，然后运行`pytest --cache-show nums`查看此时nums的值为1，再运行`pytest test_funcs.py::test_cache`，然后再`pytest --cache-show nums`，此时就变成2了。

#### `capsys`

有时候想测试`print`出来的值是不是符合预期，此时可以使用`capsys`这个`fixture`。它的`readouterr`方法返回一个元组，分别是`stdout`和`stderr`的内容，举个例子：
```python
def test_print(capsys):
    say_hello()
    out, err = capsys.readouterr()
    assert out == 'hello world\n'
    assert err == ''

    
def say_hello():
    print('hello world')
```
注意，`python`的`print`函数会自动在字符串后面加一个换行符`'\n'`，测试的时候需要加上去。  
默认清空下，`pytest`会捕获所有的`stdout`和`stderr`，因此测试用例里面的`print`不会打印任何东西到屏幕。如果你想禁用捕获，有两种方法：
1. 使用命令行参数`-s`，这个是会话级别，整个会话期间捕获都会被禁用。
2. 使用`capsys.disabled()`上下文管理器，可以临时让输出绕过默认的输出捕获机制。
```python
def test_always_print(capsys):
    with capsys.disabled():
        print('always print hello world!')
    print('disabled hello world!')
```
不加`-s`参数的话，可以看到`'always print hello world!'`，但是`'disabled hello world!'`不会显示。

#### `monkeypatch`

`monkeypatch`是个很有意思的功能，它可以临时修改实例的方法，属性，可以设置环境变量等等，然后在测试完成以后又恢复原样。有以下几个方法，其中，`raising`用于指示在属性或者字典的条目或者环境变量不存在时抛出异常：
- `setattr(target, name, value=<notset>, raising=True)`：设置一个属性，举个例子：
```python
def test_monkey_patch(monkeypatch):
    import os
    print(os.path.abspath('.'))
    monkeypatch.setattr(os.path, "abspath", lambda x: "D:\\")
    print(os.path.abspath("."))
```
可以看到，`os.path.abspath`的返回结果已经变成了"D:\"。默认情况下，`raising=True`,对于`target`不存在的属性会抛出错误，比如，你敲错了，`"abspath"`写成`"abxpath"`，如果你设置为`raising=False`，注意，此时不会抛出异常，程序会照常执行，而`os.path`此时却多了一个`abxpath`的属性。
- `delattr(target, name=<not set>, raising=True)`：删除一个属性。
- `setitem(dic, name, value)`：设置字典中的一条记录。
- `delitem(dic, name, raising=True)`：删除字典中的一条记录。
- `setenv(name, value, prepend=None)`：设置一个环境变量。
- `delenv(name, raising=True)`：删除一个环境变量。
- `syspath_prepend(path)`：将路径`path`加入`sys.path`并放在最前面。
- `chdir(path)`：改变当前的工作目录。

#### `doctest_namaspace`

这个主要是能够使用`--doctest-modules`运行`doctest`测试用例时，共享一个命名空间。假设有一个模块名为`mymath`：
```python
import mymath

def test_doctest_namespace(doctest_namespace):
    doctest_namespace['mm'] = mymath
```
这样，使用`pytest`加`--doctest_modules`运行`doctest`测试用例的时候，都可以使用`mm`这个变量来表示`mymath`模块。

#### `recwarn` 

用来检查代码里的`warning`消息，就像一个告警的信息列表，每个告警信息都有`category`，`message`，`filename`，`lineno`四个属性。除了使用`recwarn`这个`fixture`，也可以使用`pytest.warns()`上下文管理器：
```python
def warnfunc():
    warnings.warn("the func is going to be deprecated!", DeprecationWarning)

   
def test_recwarn(recwarn):
    warnfunc()
    assert len(recwarn) == 1
    w = recwarn.pop()
    assert w.category == DeprecationWarning
    assert str(w.message) == "the func is going to be deprecated!"
    

def test_pytest_warns():
    with pytest.warns(None) as warning_list:
        warnfunc()
    
    assert len(warning_list) == 1
    w = warning_list.pop()
    assert w.category == DeprecationWarning
    assert str(w.message) == "the func is going to be deprecated!"
```
其中`pytest.warns`是对`recwarn`的一个包装，一般情况下是这样用的：
```python
def test_pytest_warns():
    with pytest.warns(DeprecationWarning, match="deprecated"):
        warnfunc()
```
它返回的是一个`_pytest.recwarn.WarningsChecker`对象，注意`pyest.warns`的参数，函数签名是`with warns(expected_warning: Exception[, match])`，要注意的是，如果要像上面的例子一样，想返回一个列表，必须传入`None`，否则会报错。

#### `request`

`request`这个`fixture`不是很好理解，它主要返回函数内省的一些信息，可以用在测试函数或者`fixture`函数上。比如前面提到过的`fixture`参数化，`repuest.param`返回的是传递给`data`这个`fixture`函数的参数。再举个例子：
```python
@pytest.fixture(scope='module')
def r(request):
    return request.node.nodeid


def test_request(r):
    print(r)
```
`request`表示`r`这个`fixture`函数所在节点的字符串表示。`request`的各种属性点[这里](https://docs.pytest.org/en/latest/reference.html?highlight=pytestconfig#request)

## `pytest`的配置

比如想增加命令行选项，指定测试目录等等修改`pytest`的默认行为方式，有两种方式：
1. 使用`hook`函数，写在`confest.py`文件中。
2. 使用`pytest.ini`配置文件，如果同时使用`tox`库，也可以把`pytest`的配置写在`tox.ini`中，另外还有一个`setup.cfg`文件。  

`pytest`配置选项有很多，[点这里](https://docs.pytest.org/en/latest/reference.html#configuration-options)查看官方文档。

### 通过`pytest.ini`配置文件改变默认行为

#### `pytest.ini`文件放在哪里

`pytest`配置文件顺序是：`pytest.ini`，`tox.ini`和`setup.cfg`。它会从所有测试目录的共同目录下开始寻找`pytest.ini`，然后一直到根目录，比如目录结构是这样的：
```
E:.
│  pytest.ini      
├─test1
│  └─test_in_test1.py          
└─test2
    └─test_in_test2.py
```
`pytest.ini`只能放在`test1`和`test2`的共同目录，也就是`E`盘根目录下，不能放在`test1`或者`test2`目录下。如果`E`盘还有上级目录，放在上级目录也可以。

#### 添加默认命令行选项

可以使用`addopts`选项添加默认的命令行选项：
```ini
[pytest]
addopts = -v --tb=short
```
这样，每次使用`pytest`运行测试用例的时候，会自动添加`-v --tb=short`选项

#### 注册标记

可以使用`pytest -m <marker>`来选择运行使用`@pytest.mark.<marker>`装饰器装饰了的测试用例，但是，如果`<marker>`没有在`pytest.ini`文件中注册的话，默认会发出`warning`警告，注意，此时虽然会有警告，但是仍然会运行测试用例。可以在`pytest.ini`文件中进行注册，来避免发出警告，格式如下：
```ini
[pytest]
markers =
    smoke: Smoke is a customized marker.
    fire: fire is a customized marker.
```
此时，运行`pytest -m smoke`，不会再有警告信息出现。

另外，如果在写装饰器的时候，不小心写错了，比如本来想用`@pytest.mark.smoke`装饰测试用例的，不小心写成`@pytest.mark.smkoe`，此时可以用`--strict-markers`命令行，此时不会发出警告，而是直接抛出`Error`错误。注意，注册`marker`都是针对`@pytest.mark.<marker>`装饰的测试用例而言，而不是命令行命令`pytest -m <marker>`中的`<marker>`，不管注不注册，`pytest`都是根据`pytest -m <marker>`中的`<marker>`标记去查找测试用例。

#### 指定`pytest`忽略某些目录或者指定测试目录

默认`pytest`会搜索当前目录下的所有子目录，可以通过`norecursedirs`选项指定某些要忽略的文件夹，格式如下：
```ini
norecursedirs = .* venv src *.egg dist build
```
`.*`表示任何以`.`开头的文件夹。书上说`norecursedirs`的默认设置是`.* build dist CVS _darcs {arch} *.egg`，目前还没有找到在哪里可以看到其默认的配置。

另外一个选项是`testpaths`，告诉`pytest`只搜索某些目录，格式如下：
```ini
testpaths = tests
```
这样，`pytest`只会搜索`tests`目录。

#### 更改测试搜索规则

`python_classes`，`python_files`，`python_functions`分别修改三个不同级别的搜索规则，比如，默认情况下，只搜索`Test`开头或者结尾的类，可以通过`python_classes`修改，格式如下：
```ini
[pytest]
python_classes: *Suite *Test Test*
```

### 通过`hook`钩子函数改变默认行为

除了使用`pytest.ini`配置文件，还可以使用`hook`钩子函数改变默认行为，钩子函数都写在`conftest.py`文件内，查看所有钩子函数[点这里](https://docs.pytest.org/en/latest/reference.html#hooks)，常用的如下：

#### 添加默认命令行选项

#### 和`pytest.ini`有什么区别 

如果只是在本地自己使用的话，两者没有太大区别，`pytest.ini`方式还更简单一点。但是如果想把修改最后作成插件来发布，则只能使用`hook`钩子的方法。

## pytest-mock

- [pytest中使用mock](https://note.qidong.name/2018/02/pytest-mock/)
- [pytest: How to mock in Python](https://changhsinlee.com/pytest-mock/)
- [Mocking functions Part I | Better Unit Testing in Python with pytest-mock](https://medium.com/analytics-vidhya/mocking-in-python-with-pytest-mock-part-i-6203c8ad3606)
- [Mocking Functions Part II | Writing Better Tests in Python with pytest-mock](https://medium.com/@durgaswaroop/writing-better-tests-in-python-with-pytest-mock-part-2-92b828e1453c)

## 坑与问题收集

### 如何`monkeypatch`内置函数`datetime.datetime.now()`?

#### 问题现象

最近遇到这样的一个问题，想要测试的一个函数内部使用了`datetime.datetime.now()`，使用`monkeypatch`报错，代码如下：
```python
monkeypatch.setattr(datetime.datetime,"now", nowfunc)
```
提示：`TypeError: can't set attributes of built-in/extension type 'datetime.datetime'`。

#### 原因及解决方法

`pytest`不能对用`C`实现的许多扩展类型的属性进行`monkeypatch`，最简单的方法是建立一个子类，代码如下：
```python
class patched_datetime(datetime.datetime): 
    pass

monkeypatch.setattr(patched_datetime, "now", nowfunc)
datetime.datetime = patched_datetime
```
另外，在`stackoverflow`上有人问了同样的问题，给出的解决方案如下，直接`monkeypatch`了`datetime.datetime`类：
```python
import datetime
import pytest

FAKE_TIME = datetime.datetime(2020, 12, 25, 17, 5, 55)

@pytest.fixture
def patch_datetime_now(monkeypatch):

    class mydatetime:
        @classmethod
        def now(cls):
            return FAKE_TIME

    monkeypatch.setattr(datetime, 'datetime', mydatetime)


def test_patch_datetime(patch_datetime_now):
    assert datetime.datetime.now() == FAKE_TIME
```

### 测试文件名冲突怎么办？

如果不同的文件夹有相同名称的测试文件，此时会产生冲突，如下的文件结构：
```
E:.
│  pytest.ini      
├─test1
│  └─test_func.py          
└─test2
    └─test_func.py
```
单独运行其中一个都没问题，比如`pytest test1`或者`pytest test2`，但是使用`pytest`都运行的时候，会提示`Error`错误。解决方法是在每一个测试文件夹内加一个`__init__.py`文件，这样就可以正确的识别了。

### 如何在测试文件中导入上层目录的模块？

如下的文件结构，如果在`test_func.py`中要导入`module`，直接`import module`是不行的，会提示模块`module`找不到。
```
E:.
│  module.py 
└─tests
    └─test_func.py
```
解决办法是在`tests`文件夹下加一个`__init__.py`文件就可以了。但是注意，这种方法`pytest`可以识别，但是如果作为普通的`python`文件，还是提示不能导入，因为`pytest`做了一些额外的动作，如果加了`__init__.py`文件，会把测试文件所在目录的上级目录加入`sys.path`。  
如果是普通的`python`文件，则需要用下面的方式才能导入上级目录的模块：
```
sys.path.append((os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))))
```
具体可以查看`python私房手册-python的坑及技巧收集`。